<p>Regime switching Time-Varying-Parameter model in Statsmodels has the following specification (TVP with heteroskedastic disturbances):</p>
<p>$$ y_t = \beta_{1t} x_{1t} + \beta_{2t} x_{2t} + ... +
        \beta_{kt} x_{kt} + e_t \\
        \beta_{it} = \beta_{i,t-1} + v_{it} \\
        e_t \sim N(0, \sigma_{S_t}^2) \\
        v_{it} \sim N(0, \sigma_{i,S_t}^2) $$</p>
<p>where $\beta_{it}$ are time varying parameters and $x_{it}$ are
    exogenous variables. Observation and TVP transition error terms have a switching
    variance - $\sigma_{S_t}^2$ and $\sigma_{i,S_t}^2$ respectively.
    $S_t$ is a regime at the moment $t$.</p>
<p>Presented code uses this model for modelling U.S. monetary growth uncertainty (chapter 5.5.1 of Kim and Nelson, 1999).</p>
<p>See also http://econ.korea.ac.kr/~cjkim/MARKOV/programs/tvpmrkf.opt - the same model implementation used for testing.</p>

In [ ]:
"""
Kim's (1993) time-varying-parameter model with heteroskedastic disturbances
for U.S. monetary growth uncertainty (chapter 5.5.1 of Kim and Nelson, 1999).

Author: Valery Likhosherstov
License: Simplified-BSD
"""

import numpy as np
from statsmodels.tsa.statespace.regime_switching.api import SwitchingTVPModel
from statsmodels.tsa.statespace.regime_switching.tests.results import results_kim1993

dtype = np.float64

true = results_kim1993.tvpmrkf
start = true['start']

# Model attributes
k_regimes = 2
k_exog = 5

# Preparing data

endog = np.array(true['data']['m1'], dtype=dtype)

exog = np.zeros((endog.shape[0], k_exog), dtype=dtype)

exog[:, 0] = 1
exog[:, 1] = true['data']['dint']
exog[:, 2] = true['data']['inf']
exog[:, 3] = true['data']['surp']
exog[:, 4] = true['data']['m1lag']

# Instantiate the model
model = SwitchingTVPModel(k_regimes, endog, exog, dtype=dtype, loglikelihood_burn=start)

# Set model initial states
model.initialize_known(np.zeros(k_exog, dtype=dtype), np.identity(k_exog, dtype=dtype) * 100)

In [ ]:
# Fit the model from start parameters, used by authors
results = model.fit(start_params=np.array(true['start_parameters'], dtype=dtype))

# Print results summary
print(results.summary())

In [ ]:
%matplotlib inline

# Plot time varying parameters
plot = results.plot_coefficient(variables=range(k_exog), figsize=(8,10))

In [ ]:
# Fit the model, using non-switching start feature
results = model.fit(fit_nonswitching_first=True)

# Print results summary
print(results.summary())

<h1>References</h1>
<p>Kim, Chang-Jin, and Charles R. Nelson. 1999.<br>
"State-Space Models with Regime Switching: Classical and Gibbs-Sampling Approaches with Applications".<br>
MIT Press Books. The MIT Press.</p>